In [1]:
from Process import PlateReader
import pandas as pd
import matplotlib.pyplot as plt

### Step 1: Load Sample Information
* Ideally will be in the form of a dataframe. This dataframe should be standardized in the original documentation where we originally made the sample. 
* Either load locally or pull from the google team drive (still working on fixing pulling from Drive)

In [2]:
sample_info = pd.read_csv(r"C:\Users\Edwin\Desktop\OT2-DOE\PlanPrepareProcess\Temp or Working\06_28_21_Octadecene_Info")
sample_info

,UID,Labware,Slot,Well,ODE concentration wtf,SDS concentration wtf,ethanol concentration wtf,water concentration wtf,ODE amount mass g,ODE amount volume uL,...,SDS amount volume uL,ethanol amount mass g,ethanol amount volume uL,water amount mass g,water amount volume uL,ODE-ethanol-stock amount volume uL,SDS-ethanol-stock amount volume uL,ethanol-stock amount volume uL,water-stock amount volume uL,Total Volume
0,S2_A1_07-02-2021,Abgene 96 Well Plate 2200 ÂµL,2,A1,0.000,0.0,0.100000,0.900000,0.000,0.000000,...,0.0,0.100000,126.694539,0.900000,900.000000,0.0,0.0,126.742712,900.000000,1026.742712
1,S2_A2_07-02-2021,Abgene 96 Well Plate 2200 ÂµL,2,A2,0.000,0.0,0.177778,0.822222,0.000,0.000000,...,0.0,0.177778,225.234737,0.822222,822.222222,0.0,0.0,225.320377,822.222222,1047.542600
2,S2_A3_07-02-2021,Abgene 96 Well Plate 2200 ÂµL,2,A3,0.000,0.0,0.255556,0.744444,0.000,0.000000,...,0.0,0.255556,323.774934,0.744444,744.444444,0.0,0.0,323.898043,744.444444,1068.342487
3,S2_A4_07-02-2021,Abgene 96 Well Plate 2200 ÂµL,2,A4,0.000,0.0,0.333333,0.666667,0.000,0.000000,...,0.0,0.333333,422.315132,0.666667,666.666667,0.0,0.0,422.475708,666.666667,1089.142374
4,S2_A5_07-02-2021,Abgene 96 Well Plate 2200 ÂµL,2,A5,0.000,0.0,0.411111,0.588889,0.000,0.000000,...,0.0,0.411111,520.855329,0.588889,588.888889,0.0,0.0,521.053373,588.888889,1109.942262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,S2_H7_07-02-2021,Abgene 96 Well Plate 2200 ÂµL,2,H7,0.001,0.0,0.488889,0.510111,0.001,1.267427,...,0.0,0.488889,619.395526,0.510111,510.111111,200.0,0.0,420.898465,510.111111,1131.009576
91,S2_H8_07-02-2021,Abgene 96 Well Plate 2200 ÂµL,2,H8,0.001,0.0,0.566667,0.432333,0.001,1.267427,...,0.0,0.566667,717.935724,0.432333,432.333333,200.0,0.0,519.476130,432.333333,1151.809463
92,S2_H9_07-02-2021,Abgene 96 Well Plate 2200 ÂµL,2,H9,0.001,0.0,0.644444,0.354556,0.001,1.267427,...,0.0,0.644444,816.475921,0.354556,354.555556,200.0,0.0,618.053795,354.555556,1172.609351
93,S2_H10_07-02-2021,Abgene 96 Well Plate 2200 ÂµL,2,H10,0.001,0.0,0.722222,0.276778,0.001,1.267427,...,0.0,0.722222,915.016118,0.276778,276.777778,200.0,0.0,716.631460,276.777778,1193.409238


### Step 2: Load UV-Vis Data
* ***If loading from plate reader***: Once the plate has been read, allow for exporting to an excel sheet and reformat for easy upload and merging.
    * The plate reader should automatically ask you if you would like to export to excel. When running multiple plates you can just keep selecting export and it will automatically add the new run as an additional sheet to the originally generated excel sheet. 
    * To format for easy loading:
        1. Create a new sheet in the same exported excel file and name it something simple (i.e. sheet1). 
        2. Copy the data from the sheet and add it to the newly created sheet. Make sure to paste in the data as "only values". 
        3. Repeat for every single exported sheet (i.e. every plate)
         
* Sometimes when using the plate reader the scanning protocol scans the whole plate versus only the wells of interest. This can be changed, however it is common to forget. If you do forget to do this, make sure to trim the dataframe to match the same length/order of wells as the sample info loaded in Step 1.

* ***If loading from single cuvette reader***: 
    * Still documenting.

***Step 2a:*** Provide the path of the excel sheet for the plate reader. With this you will also need to provide the **Sheet Name** of for correct excel sheet to be extracted. Since multiple plates are expected to be used at once, provide the Sheet names as a ***list of string names***. This will return a list of dataframes of each plate in the order the sheet names provided.

In [4]:
path = r"C:\Users\Edwin\Downloads\06_28_21_Octadecene.xlsx"
plate_names = ['Sheet1']
plate_dfs = PlateReader.extract_plates(path, plate_names)

***Step 2b:*** With the dataframes from ***Step2a***, given that all the plates read originally were done using the same protcol, then we can merge these dataframes into one. The only thing we need to edit is when originally formatting the excel sheets, since we copied the wavelength row in each sheet it only needs to be present once. A way to avoid to this is to copy it only in the first plate sheet you are using. 

Should be length of sample_info + 1 (will remove the wavelength column shortly).

In [5]:
merged_df= PlateReader.merge_wavelength_dfs(plate_dfs)

***Step 2c:*** Sometimes when samples are extremely turbid or have some sort of material like foam or cream the plate reader absorbance is overflowed and returns as 'OVRFLW'. This will prevent you from handling the entire set of data, so what can be done is replace the 'OVRFLW' values with a holder number. It is reccomended you select a very high holder number so you can track and isolate it easier if needed. 

In [7]:
merged_dfi = PlateReader.detect_ovflw(merged_df, holder = float('nan'))
merged_dfi

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
Wavelength,300.000,305.000,310.000,315.000,320.000,325.000,330.000,335.000,340.000,345.000,...,655.000,660.000,665.000,670.000,675.000,680.000,685.000,690.000,695.000,700.000
A1,0.287,0.205,0.176,0.157,0.136,0.119,0.106,0.088,0.076,0.069,...,0.038,0.038,0.038,0.038,0.038,0.039,0.039,0.039,0.039,0.039
A2,0.282,0.202,0.173,0.155,0.134,0.118,0.105,0.088,0.076,0.069,...,0.037,0.037,0.037,0.037,0.037,0.037,0.037,0.037,0.037,0.038
A3,0.281,0.201,0.172,0.154,0.134,0.117,0.105,0.088,0.076,0.070,...,0.038,0.038,0.037,0.038,0.037,0.037,0.037,0.037,0.037,0.038
A4,0.283,0.203,0.174,0.156,0.135,0.119,0.105,0.089,0.077,0.069,...,0.040,0.040,0.040,0.040,0.040,0.039,0.039,0.039,0.039,0.040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H8,1.081,0.998,0.963,0.940,0.916,0.896,0.879,0.861,0.846,0.835,...,0.647,0.646,0.644,0.642,0.640,0.638,0.636,0.634,0.632,0.630
H9,0.795,0.711,0.677,0.656,0.633,0.615,0.600,0.582,0.568,0.558,...,0.429,0.428,0.426,0.425,0.424,0.423,0.422,0.421,0.420,0.419
H10,0.551,0.469,0.436,0.415,0.392,0.374,0.359,0.341,0.326,0.317,...,0.225,0.224,0.224,0.223,0.223,0.222,0.222,0.221,0.221,0.221
H11,0.267,0.192,0.164,0.147,0.128,0.114,0.102,0.087,0.075,0.069,...,0.038,0.038,0.038,0.038,0.037,0.037,0.037,0.037,0.037,0.038


# Step 3: Merge with Sample Info Dataframe

The first step to merge to sample info dataframe is to format the plate reader dataframe to have the wavelengths as the column names versus as an another row. We can do this by isolating the Wavelength row, adding the appropiate unit the string of each element of the wavlength row and reassigning this new list as the column names. We finalize this by dropping the wavlength row.

In [8]:
platereader_df = PlateReader.rehead_wavelengths(merged_dfi)

platereader_df

,300.0nm,305.0nm,310.0nm,315.0nm,320.0nm,325.0nm,330.0nm,335.0nm,340.0nm,345.0nm,...,655.0nm,660.0nm,665.0nm,670.0nm,675.0nm,680.0nm,685.0nm,690.0nm,695.0nm,700.0nm
A1,0.287,0.205,0.176,0.157,0.136,0.119,0.106,0.088,0.076,0.069,...,0.038,0.038,0.038,0.038,0.038,0.039,0.039,0.039,0.039,0.039
A2,0.282,0.202,0.173,0.155,0.134,0.118,0.105,0.088,0.076,0.069,...,0.037,0.037,0.037,0.037,0.037,0.037,0.037,0.037,0.037,0.038
A3,0.281,0.201,0.172,0.154,0.134,0.117,0.105,0.088,0.076,0.070,...,0.038,0.038,0.037,0.038,0.037,0.037,0.037,0.037,0.037,0.038
A4,0.283,0.203,0.174,0.156,0.135,0.119,0.105,0.089,0.077,0.069,...,0.040,0.040,0.040,0.040,0.040,0.039,0.039,0.039,0.039,0.040
A5,0.303,0.217,0.186,0.167,0.144,0.126,0.111,0.093,0.079,0.071,...,0.039,0.039,0.039,0.039,0.039,0.039,0.039,0.039,0.039,0.039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H8,1.081,0.998,0.963,0.940,0.916,0.896,0.879,0.861,0.846,0.835,...,0.647,0.646,0.644,0.642,0.640,0.638,0.636,0.634,0.632,0.630
H9,0.795,0.711,0.677,0.656,0.633,0.615,0.600,0.582,0.568,0.558,...,0.429,0.428,0.426,0.425,0.424,0.423,0.422,0.421,0.420,0.419
H10,0.551,0.469,0.436,0.415,0.392,0.374,0.359,0.341,0.326,0.317,...,0.225,0.224,0.224,0.223,0.223,0.222,0.222,0.221,0.221,0.221
H11,0.267,0.192,0.164,0.147,0.128,0.114,0.102,0.087,0.075,0.069,...,0.038,0.038,0.038,0.038,0.037,0.037,0.037,0.037,0.037,0.038


Next we reset both indexes of the two dataframes we are to merge and concatnate them together. We drop and reset indexes to ensure any past dependency is forgotten, so you must make sure your sample info dataframe and plate reader dataframe are ordered correctly where row 1...n of one dataframe referes to row 1...n of the other. 

In [9]:
spectras = platereader_df
sample_info.reset_index(drop=True, inplace=True)
spectras.reset_index(drop=True, inplace=True)
combined_df = pd.concat([sample_info, spectras], axis = 1)[:len(sample_info)] # switch to the dicionary version since you could store a unblanked, blanked and a blanker in one file.
combined_df

,UID,Labware,Slot,Well,ODE concentration wtf,SDS concentration wtf,ethanol concentration wtf,water concentration wtf,ODE amount mass g,ODE amount volume uL,...,655.0nm,660.0nm,665.0nm,670.0nm,675.0nm,680.0nm,685.0nm,690.0nm,695.0nm,700.0nm
0,S2_A1_07-02-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,A1,0.000,0.0,0.100000,0.900000,0.000,0.000000,...,0.038,0.038,0.038,0.038,0.038,0.039,0.039,0.039,0.039,0.039
1,S2_A2_07-02-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,A2,0.000,0.0,0.177778,0.822222,0.000,0.000000,...,0.037,0.037,0.037,0.037,0.037,0.037,0.037,0.037,0.037,0.038
2,S2_A3_07-02-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,A3,0.000,0.0,0.255556,0.744444,0.000,0.000000,...,0.038,0.038,0.037,0.038,0.037,0.037,0.037,0.037,0.037,0.038
3,S2_A4_07-02-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,A4,0.000,0.0,0.333333,0.666667,0.000,0.000000,...,0.040,0.040,0.040,0.040,0.040,0.039,0.039,0.039,0.039,0.040
4,S2_A5_07-02-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,A5,0.000,0.0,0.411111,0.588889,0.000,0.000000,...,0.039,0.039,0.039,0.039,0.039,0.039,0.039,0.039,0.039,0.039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,S2_H7_07-02-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,H7,0.001,0.0,0.488889,0.510111,0.001,1.267427,...,0.876,0.874,0.871,0.869,0.866,0.864,0.861,0.859,0.857,0.855
91,S2_H8_07-02-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,H8,0.001,0.0,0.566667,0.432333,0.001,1.267427,...,0.647,0.646,0.644,0.642,0.640,0.638,0.636,0.634,0.632,0.630
92,S2_H9_07-02-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,H9,0.001,0.0,0.644444,0.354556,0.001,1.267427,...,0.429,0.428,0.426,0.425,0.424,0.423,0.422,0.421,0.420,0.419
93,S2_H10_07-02-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,H10,0.001,0.0,0.722222,0.276778,0.001,1.267427,...,0.225,0.224,0.224,0.223,0.223,0.222,0.222,0.221,0.221,0.221


# Step 4: Format Dataframe as csv and Upload to Drive.
We simply call a method of the dataframe while ensuring to drop the index (again to remove and previous dependency). 

***If uploading from desktop***: Just grab and upload like any other file.

***If uploading from jupyter notebook***: Save the csv anywhere with a name that is not relevant, reccomended is something like "Temp File" as you will be uploading it to google drive under a different name. Then use the Gdrive module of functions from the Prepare package. Read the README in this package to understand how to use it, it will require a certain setup of credentials. 


In [10]:
temp_hold_path = r"C:\Users\Edwin\Desktop\OT2-DOE\PlanPrepareProcess\Temp or Working\06_28_21_ODE_SDS_Unblanked_Abs"
combined_df_csv = combined_df.to_csv(temp_hold_path, index=False)